In [1]:
import numpy as np
import subprocess
import os
import shap
import scipy.stats as stats
from multiprocessing import Pool
from utils.rerank import write_average, rerank_ndcg, rerank_matrix,write_tau
from utils.readdata import get_microsoft_data, rewrite
from utils.separate_set import separate_set
from utils.explainer_tools import rand_row, get_set_cover, evaluate, get_pairsname, get_rankedduculist

In [ ]:
def get_set_cover(shap_values):
    """
    get scores of the samples of this query and rank them according to the scores,
    we select the 10_top important features
    :param shap_values:
    :param threshold_flag: 0: simple sum, 1: set the threshold to be 2 means, 2: mean - 3*shap_values_std
    :return:
    """
    shap_values =np.array([shap_values])
    # shap_values[shap_values < 0] = 0
    sumvalue = np.sum(shap_values,axis=1)
    mean =  sumvalue/shap_values.shape[1]
    shap_values_std = np.std(shap_values,ddof=1, axis=1)
    top_k = 10
    top_k_idx = []
    for i in range(top_k):
        feature_index=((-sumvalue).argsort())[0][0]
        top_k_idx.append(feature_index)
        a = np.array([[shap_values[0][i][1] for i in range(len(shap_values[0]))]])
        bigindex = list(((-a).argsort())[0][:3])
        #bigindex = list(np.where(shap_values[0][:,feature_index]>threshold[0][feature_index]))
        shap_values[0][:,feature_index] = 0
        shap_values[0][bigindex] = 0
        sumvalue = np.sum(shap_values, axis=1)
        if np.all(sumvalue == 0): break
    return top_k_idx, mean

In [ ]:
def get_set_cover(shap_values, threshold_flag):
    """
    get scores of the samples of this query and rank them according to the scores,
    we select the 10_top important features
    :param shap_values:
    :param threshold_flag: 0: simple sum, 1: set the threshold to be 2 means, 2: mean - 3*shap_values_std
    :return:
    """
    shap_values =np.array([shap_values])
    sumvalue = np.sum(shap_values,axis=1)   
    mean =  sumvalue/len(shap_values)
    temp_shap = shap_values[0].copy()
    min_value = np.min(shap_values, axis=1)
    for x in range(0, temp_shap.shape[0]):
        for y in range(0, temp_shap.shape[1]):
            if temp_shap[x,y] <= 0:
                temp_shap[x,y] = 0
    sumpositive = np.sum(temp_shap, axis=0)
    positive_mean = sumpositive/len(shap_values)
    shap_values_std = np.std(temp_shap,ddof=1, axis=0)
    top_k = 10
    if threshold_flag == 0:
        top_k_idx=((-sumvalue).argsort())[0][0:top_k]
        return top_k_idx, mean
    elif threshold_flag== 1:
        threshold = [[0 for i in range(46)]]
        #threshold = [positive_mean - 3*shap_values_std]
    elif threshold_flag == 2:
        #threshold = [[i/2 for i in positive_mean]]
        threshold = [positive_mean]
    top_k_idx = []
    for i in range(top_k):
        feature_index=((-sumvalue).argsort())[0][0]
        top_k_idx.append(feature_index)
        bigindex = list(np.where(shap_values[0][:,feature_index]>=threshold[0][feature_index]))
        shap_values[0][:,feature_index] = min_value[0][feature_index]
        shap_values[0][bigindex] = min_value[0][feature_index]
        sumvalue = np.sum(shap_values, axis=1)
    return top_k_idx, mean

In [2]:
def score(X):
    """
    The first if branch is training data, the next is for the single test data. First calling the subprocess of ranklib
    to get the scores, then rerank the scorefile according the original index. We also have to delete the produced
    files which used by the subprocess.
    :param X: input feature matrix
    :return: scores of q-d pairs
    """
    A = []
    scorefile_path = temp_path + 'scorefile_newmatrix_{}.txt'.format(tmp_test_y_query[0].split(':')[-1].split()[0])
    restore_path = temp_path + 'restore_newmatrix_{}.txt'.format(tmp_test_y_query[0].split(':')[-1].split()[0])
    rewrite(X, tmp_test_y_query, tmp_test_Query, restore_path)
    args = ['java', '-jar', 'RankLib-2.12.jar', '-rank', restore_path, '-load', model,
            '-indri', scorefile_path]
    subprocess.check_output(args, stderr=subprocess.STDOUT)

    # rerank the scorefile according the original index
    scorefile_data = ''.join(sorted(open(scorefile_path), key=lambda s: s.split()[1], reverse=False))
    with open(scorefile_path, 'w') as f:
        f.write(scorefile_data)
    with open(scorefile_path, 'r') as f:
        for line in f:
            A.append(float(line.split()[-2]))

    # reset the index to be original otherwise can not get the right NDCG
    restore_context = open(restore_path, 'r').readlines()
    with open(restore_path, 'w') as f:
        for lineindex in range(len(restore_context)):
            split = restore_context[lineindex].split()
            split[1] = 'qid:{}'.format(tmp_test_y_query[0].split(':')[-1].split()[0])
            newline = ''
            for i in range(len(split)):
                newline += (split[i] + ' ')
            f.write(newline + '\n')
    A = np.array(A)
    return A


def loop_query(query_index):
    """
    loop for a query, get scores of the samples of this query and rank them according to the scores
    :param query_index: the index of query
    :return: ranklist file, matrix file, delta NDCG file
    """
    # get data for this query
    global tmp_test_data
    global tmp_test_y_query
    global tmp_test_Query
    tmp_test_data =test_data[query_index]
    tmp_test_y_query = test_y_query[query_index]
    tmp_test_Query = test_Query[query_index]
    query_id = tmp_test_y_query[0].split(':')[-1].split()[0]

    # calculate the scores for the q-d pairs
    restore_path = temp_path +  'restore_newmatrix_{}.txt'.format(query_id)
    scorefile_path = temp_path + 'scorefile_newmatrix_{}.txt'.format(query_id)
    scores = score(tmp_test_data).reshape(-1, 1)


    # reranking the test_data according to the scores and get the list of ranking
    test_data_score = np.append(tmp_test_data,scores,axis=1)
    ranked_test_data = np.array((test_data_score[(-test_data_score[:,-1]).argsort()])[:,:-1])
    rankedduculist1 = get_rankedduculist(scores, query_index, q_d_len)
    NDCG_before = evaluate(model, restore_path)

    # get pairsname
    pairnumbers = 20
    pairsname = get_pairsname(ranked_test_data, pairnumbers)
    
    def get_score_matrix(feature_matrix):
        changed_list = []
        for i in range(feature_matrix.shape[0]):
            temp =  feature_matrix[i].copy()
            for m in range(46):
                temp2 = temp.copy()
                temp2[m] = expected_value[m]
                changed_list.append(temp2)
        changed_list = np.array(changed_list)
        with open(temp_path+'changed_list_newmatrix{}.txt'.format(query_index),'w') as f:
            for i in range(feature_matrix.shape[0]*46):
                line = ""
                line += "0 qid:{} ".format(str(i))
                for j in range(len(changed_list[i])):
                    line += ((str(j+1))+":"+str(changed_list[i][j])+" ")
                line += '#docid = GX008-86-4444840 inc = 1 prob = 0.086622 ' + "\n"
                f.write(line)
        args = ['java', '-jar', 'RankLib-2.12.jar', '-rank', temp_path+'changed_list_newmatrix{}.txt'.format(query_index), '-load', model,
                '-indri', temp_path+'changed_list_new_matrix_score{}.txt'.format(query_index)]
        subprocess.check_output(args, stderr=subprocess.STDOUT)
        A = ''.join(sorted(open(temp_path+'changed_list_new_matrix_score{}.txt'.format(query_index)), key=lambda s: int(s.split()[0]), reverse=False))
        with open(temp_path+'changed_list_new_matrix_score{}.txt'.format(query_index),'w') as f:
            f.write(A)
        changed_list_score = []
        with open(temp_path+'changed_list_new_matrix_score{}.txt'.format(query_index),'r') as f:
            for line in f:
                changed_list_score.append(float(line.split()[-2]))
        changed_list_score =  [changed_list_score[i:i + 46] for i in range(0, len(changed_list_score), 46)]   
        os.remove(os.path.join(temp_path, 'changed_list_newmatrix{}.txt'.format(query_index)))
        os.remove(os.path.join(temp_path, 'changed_list_new_matrix_score{}.txt'.format(query_index))) 
        return changed_list_score
        
    # create the matrix
    score_values = get_score_matrix(ranked_test_data)
    matrix = []
    for i in range(len(pairsname)):
        index1 = int(pairsname[i][1])
        index2 = int(pairsname[i][-1])
        row = [(score_values[index1-1][j]-score_values[index2-1][j]) for j in range(46)]
        matrix.append(row)


    def feature_k_loop(feature_number,threshold_flag):
        top_k_idx,_ = get_set_cover(matrix, threshold_flag)
        NDCG_file_name = NDCGdata_path + 'SHAP_newmatrix_{}features_threshold{}'.format(feature_number,threshold_flag) + modelname + '.txt'
        NDCG_file_matrix = NDCGdata_path + 'SHAP_newmatrix_matrix_{}features_threshold{}'.format(feature_number,threshold_flag)  + modelname + '.txt'
        ranklist_file = NDCGdata_path + 'ranklist_SHAP_newmatrix_{}features_threshold{}'.format(feature_number,threshold_flag)  + modelname + '.txt'
        features_to_change = tmp_test_data
        if len(top_k_idx)<= feature_number:
            feature_number = len(top_k_idx)
        features_to_change[:,top_k_idx[0:feature_number]] = expected_value[top_k_idx[0:feature_number]]
        restore_path = temp_path +  'restore_newmatrix_{}.txt'.format(query_id,feature_number)
        scorefile_path = temp_path + 'scorefile_newmatrix_{}.txt'.format(query_id,feature_number)
        # get scores of the changed features
        scores2 = score(features_to_change).reshape(-1,1)
        rankedduculist2 = get_rankedduculist(scores2, query_index, q_d_len)
        NDCG_after = evaluate(model, restore_path)
        delta_NDCG = float(NDCG_before) - float(NDCG_after)
        tau, p_value = stats.kendalltau(rankedduculist1, rankedduculist2)
  
        with open(NDCG_file_name, 'a') as NDCG_FILE:
            NDCG_line =  tmp_test_y_query[0].split(':')[-1]+'  ' \
                        + 'changed feature:'+ str(top_k_idx)+' '+'kendalltau='+str(round(tau,4))+ ' '+ 'pairnames:'+' '+str(pairsname) + \
                        '   ' + 'delta_NDCG ='+'  '+str(round(delta_NDCG,4))+ "\n"
            NDCG_FILE.write(NDCG_line)
        with open(NDCG_file_matrix, 'a') as matrix_FILE:
            matrix_line = 'matrix for {}'.format(tmp_test_y_query[0].split(':')[-1].split()[0]) \
                          + '  ' + str(matrix) + '  ' + "\n"
            matrix_FILE.write(matrix_line)
        with open(ranklist_file, 'a') as ranklist:
            ranklist_line = tmp_test_y_query[0].split(':')[-1] + '  ' + 'ranklist before:' + str(
                rankedduculist1) + '  ' + 'ranklist after:' + '  ' + str(rankedduculist2) + "\n"
            ranklist.write(ranklist_line)
        os.remove(scorefile_path)
        os.remove(restore_path)
                                                                
    feature_k_loop(5,threshold_flag)        
    feature_k_loop(10,threshold_flag)

In [22]:
model_path = 'model/'
model = model_path + 'LambdaMART_model.txt'


# the path of data
datapath = 'MQ2008/Fold1/'
train_path = datapath + 'train.txt'
test_path = datapath + 'test.txt'
modelname = model.split("_")[0].split("/")[-1]

# saving path and save files
NDCGdata_path = 'NDCGdata/'
temp_path = 'temp_data1/'


# get train data and test data
X_train, y_query_train, Query_train = get_microsoft_data(train_path)
X_train = np.array(X_train)
X_test, y_query_test, Query_test = get_microsoft_data(test_path)
X_test = np.array(X_test)
expected_value = np.mean(X_train, axis=0)

# separate the test set
test_data, test_y_query, test_Query, q_d_len = separate_set(y_query_test, X_test, Query_test)
expected_value

array([0.15509742, 0.13651554, 0.20418996, 0.13261472, 0.15699957,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.27906804, 0.13507181, 0.19889295, 0.1209401 , 0.28310363,
       0.14905815, 0.22061679, 0.30021336, 0.32195704, 0.14922716,
       0.54492714, 0.56017743, 0.54294727, 0.56042099, 0.16075669,
       0.13240912, 0.12051001, 0.11981521, 0.19962213, 0.15990731,
       0.15622884, 0.14974686, 0.12759748, 0.07848159, 0.07887539,
       0.07429701, 0.54698628, 0.55893672, 0.54855439, 0.56609091,
       0.43337705, 0.33050601, 0.        , 0.10266228, 0.11602421,
       0.15198914])

In [23]:

for i in range(len(pairsname)):
    index1 = int(pairsname[i][1])
    index2 = int(pairsname[i][-1])
    print(index1)
    print(index2)
    row = [(score_values[index2-1][j]-score_values[index1-1][j]) for j in range(46)]
    matrix.append(row)

1
5
1
3
1
2
1
7
2
8
2
7
2
6
2
3
2
4
3
7
3
6
3
4
4
6
4
7
4
8
5
8
5
7
5
6
6
7
7
8


In [6]:
tmp_test_data =test_data[0]
tmp_test_y_query = test_y_query[0]
tmp_test_Query = test_Query[0]
query_id = tmp_test_y_query[0].split(':')[-1].split()[0]

# calculate the scores for the q-d pairs
restore_path = temp_path +  'restore_newmatrix_{}.txt'.format(query_id)
scorefile_path = temp_path + 'scorefile_newmatrix_{}.txt'.format(query_id)
scores = score(tmp_test_data).reshape(-1, 1)


# reranking the test_data according to the scores and get the list of ranking
test_data_score = np.append(tmp_test_data,scores,axis=1)
ranked_test_data = np.array((test_data_score[(-test_data_score[:,-1]).argsort()])[:,:-1])
rankedduculist1 = get_rankedduculist(scores, 0, q_d_len)
NDCG_before = evaluate(model, restore_path)

# get pairsname
pairnumbers = 20
pairsname = get_pairsname(ranked_test_data, pairnumbers)
ranked_test_data

array([[2.64460e-02, 7.50000e-01, 7.50000e-01, 5.00000e-01, 3.64240e-02,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        2.41160e-02, 7.16563e-01, 7.02528e-01, 5.21932e-01, 3.16830e-02,
        7.91900e-03, 1.00000e+00, 5.00000e-01, 2.00000e-01, 8.17000e-03,
        7.80898e-01, 6.59859e-01, 7.22056e-01, 6.66725e-01, 4.28280e-01,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 8.76585e-01, 9.39544e-01,
        6.64658e-01, 9.53860e-01, 5.48049e-01, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 8.43365e-01, 7.27569e-01, 7.86101e-01, 7.25966e-01,
        0.00000e+00, 6.08696e-01, 0.00000e+00, 1.00000e+00, 1.00000e+00,
        2.66667e-01],
       [5.28930e-02, 1.00000e+00, 7.50000e-01, 1.00000e+00, 6.62250e-02,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        4.76340e-02, 1.00000e+00, 7.40506e-01, 1.00000e+00, 5.85390e-02,
        3.99500e-03, 5.00000e-01, 4.00000e-01, 4.00000e-01, 4.12100e-03,
        1.00000e+00, 1.00000e

In [10]:
query_index = 0
def get_score_matrix(feature_matrix):
        changed_list = []
        for i in range(feature_matrix.shape[0]):
            temp =  feature_matrix[i].copy()
            for m in range(46):
                temp2 = temp.copy()
                temp2[m] = expected_value[m]
                changed_list.append(temp2)
        changed_list = np.array(changed_list)
        print(changed_list[0:10])
        with open(temp_path+'changed_list_newmatrix{}.txt'.format(query_index),'w') as f:
            for i in range(feature_matrix.shape[0]*46):
                line = ""
                line += "0 qid:{} ".format(str(i))
                for j in range(len(changed_list[i])):
                    line += ((str(j+1))+":"+str(changed_list[i][j])+" ")
                line += '#docid = GX008-86-4444840 inc = 1 prob = 0.086622 ' + "\n"
                f.write(line)
        args = ['java', '-jar', 'RankLib-2.12.jar', '-rank', temp_path+'changed_list_newmatrix{}.txt'.format(query_index), '-load', model,
                '-indri', temp_path+'changed_list_new_matrix_score{}.txt'.format(query_index)]
        subprocess.check_output(args, stderr=subprocess.STDOUT)
        A = ''.join(sorted(open(temp_path+'changed_list_new_matrix_score{}.txt'.format(query_index)), key=lambda s: int(s.split()[0]), reverse=False))
        with open(temp_path+'changed_list_new_matrix_score{}.txt'.format(query_index),'w') as f:
            f.write(A)
        changed_list_score = []
        with open(temp_path+'changed_list_new_matrix_score{}.txt'.format(query_index),'r') as f:
            for line in f:
                changed_list_score.append(float(line.split()[-2]))
        changed_list_score =  [changed_list_score[i:i + 46] for i in range(0, len(changed_list_score), 46)]   
        os.remove(os.path.join(temp_path, 'changed_list_newmatrix{}.txt'.format(query_index)))
        os.remove(os.path.join(temp_path, 'changed_list_new_matrix_score{}.txt'.format(query_index))) 
        return changed_list_score
score_values = get_score_matrix(ranked_test_data)
score_values

[[0.15509742 0.75       0.75       0.5        0.036424   0.
  0.         0.         0.         0.         0.024116   0.716563
  0.702528   0.521932   0.031683   0.007919   1.         0.5
  0.2        0.00817    0.780898   0.659859   0.722056   0.666725
  0.42828    0.         0.         0.         0.876585   0.939544
  0.664658   0.95386    0.548049   0.         0.         0.
  0.843365   0.727569   0.786101   0.725966   0.         0.608696
  0.         1.         1.         0.266667  ]
 [0.026446   0.13651554 0.75       0.5        0.036424   0.
  0.         0.         0.         0.         0.024116   0.716563
  0.702528   0.521932   0.031683   0.007919   1.         0.5
  0.2        0.00817    0.780898   0.659859   0.722056   0.666725
  0.42828    0.         0.         0.         0.876585   0.939544
  0.664658   0.95386    0.548049   0.         0.         0.
  0.843365   0.727569   0.786101   0.725966   0.         0.608696
  0.         1.         1.         0.266667  ]
 [0.026446   0.7

[[2.84524,
  3.92822,
  4.0115,
  4.3325,
  3.79441,
  4.11688,
  4.11688,
  4.11688,
  4.11688,
  4.11688,
  4.12485,
  2.94357,
  3.72486,
  2.21735,
  3.71159,
  5.25701,
  4.36906,
  3.78187,
  4.32356,
  3.82679,
  3.51346,
  4.21754,
  3.92434,
  4.65602,
  3.56595,
  4.12943,
  4.14816,
  4.0706,
  2.8537,
  4.76159,
  4.20103,
  3.03249,
  4.14866,
  4.17192,
  4.11688,
  3.79677,
  4.33386,
  3.76099,
  3.70622,
  3.86106,
  3.15084,
  4.21947,
  4.11688,
  4.2153,
  3.75434,
  3.97384],
 [-0.42406,
  0.24355,
  0.04645,
  0.52438,
  0.17016,
  0.15184,
  0.15184,
  0.15184,
  0.15184,
  0.15184,
  0.09114,
  0.09214,
  -0.16466,
  -0.25286,
  0.23489,
  1.70934,
  0.22634,
  0.05798,
  0.06153,
  0.44274,
  -0.51297,
  -0.02082,
  -2.03029,
  -0.73854,
  -2.97037,
  0.64629,
  -0.07954,
  -0.23124,
  -0.43472,
  0.42017,
  0.14985,
  0.49028,
  0.07799,
  0.80438,
  -0.23655,
  0.34445,
  1.24927,
  -0.52757,
  -1.78843,
  -1.79946,
  -0.17892,
  0.15184,
  0.15184,
  0.24595,
  -0.01546,
  -0.18925],
 [-0.93322,
  0.25603,
  0.03352,
  0.23026,
  0.13747,
  0.03352,
  0.03352,
  0.03352,
  0.03352,
  0.03352,
  0.27869,
  0.09185,
  0.22438,
  -1.90514,
  0.01551,
  0.76518,
  0.17135,
  0.03436,
  0.66142,
  -0.51994,
  -0.07308,
  -0.00842,
  -0.93878,
  0.49468,
  -0.47126,
  -0.47701,
  0.12763,
  -0.08451,
  0.03352,
  0.13557,
  -0.09216,
  -0.1495,
  0.0647,
  0.02992,
  0.03352,
  -0.31627,
  1.1342,
  -0.48522,
  -0.79654,
  0.15183,
  -0.30168,
  0.491,
  0.03352,
  0.35765,
  0.03285,
  -0.07779],
 [-1.10207,
  -0.91017,
  -0.72336,
  -0.35082,
  -0.74585,
  -0.72336,
  -0.72336,
  -0.72336,
  -0.72336,
  -0.72336,
  -0.70795,
  -1.63008,
  -0.60772,
  -1.30009,
  -0.80097,
  0.59239,
  -0.51981,
  -0.72336,
  -0.33072,
  -0.61756,
  -1.02,
  -1.34785,
  -2.14686,
  -0.27023,
  -1.95569,
  -0.51375,
  -0.72336,
  -2.40277,
  -0.83172,
  -0.72336,
  -0.72336,
  -0.69523,
  -0.63242,
  -0.2025,
  -1.11175,
  -0.53075,
  0.59105,
  -1.81066,
  -3.37184,
  -2.00398,
  -0.59503,
  -0.66909,
  -0.72336,
  -0.85416,
  -0.6603,
  -0.71331],
 [-2.58335,
  -1.83678,
  -2.24174,
  -1.78754,
  -2.39947,
  -2.03163,
  -2.03163,
  -2.03163,
  -2.03163,
  -2.03163,
  -2.23784,
  -1.98389,
  -2.45164,
  -2.10129,
  -2.46025,
  -0.61682,
  -1.74585,
  -1.90231,
  -1.97728,
  -1.7389,
  -2.37818,
  -1.92227,
  -2.44969,
  -1.59881,
  -2.39866,
  -2.59068,
  -1.9878,
  -2.03163,
  -2.62744,
  -2.5978,
  -1.96901,
  -2.73159,
  -2.00605,
  -1.10485,
  -2.42804,
  -2.18389,
  -1.53768,
  -2.64375,
  -2.69375,
  -2.37688,
  -2.24675,
  -1.81866,
  -2.03163,
  -1.66605,
  -1.99972,
  -2.1746],
 [-4.68503,
  -3.53075,
  -3.78029,
  -3.60025,
  -4.09196,
  -3.78029,
  -3.78029,
  -3.78029,
  -3.78029,
  -3.78029,
  -3.3963,
  -3.72196,
  -3.66465,
  -5.87075,
  -3.93638,
  -3.99583,
  -3.66849,
  -3.41615,
  -5.04315,
  -4.15552,
  -3.87,
  -3.66537,
  -3.55779,
  -3.72318,
  -4.26613,
  -4.29083,
  -3.70279,
  -3.89832,
  -3.86371,
  -3.78169,
  -3.75208,
  -4.13,
  -3.67174,
  -3.69969,
  -3.78029,
  -4.00652,
  -3.47498,
  -3.61148,
  -1.62055,
  -3.04327,
  -3.57158,
  -3.91479,
  -3.78029,
  -3.35739,
  -3.98511,
  -4.2732],
 [-4.72386,
  -4.56631,
  -4.78536,
  -4.78536,
  -4.18465,
  -4.78536,
  -4.78536,
  -4.78536,
  -4.78536,
  -4.78536,
  -4.34551,
  -4.69577,
  -4.66971,
  -4.7414,
  -5.04665,
  -4.46615,
  -4.83486,
  -4.90527,
  -4.28654,
  -4.60113,
  -4.80323,
  -4.52449,
  -5.51557,
  -4.74906,
  -5.21921,
  -5.25058,
  -4.74726,
  -4.91715,
  -4.95869,
  -4.78536,
  -4.78536,
  -4.94882,
  -4.78536,
  -4.63655,
  -4.78536,
  -5.63582,
  -4.35464,
  -4.40909,
  -4.19142,
  -4.09582,
  -4.91976,
  -4.86681,
  -4.78536,
  -4.66969,
  -4.80898,
  -5.80323],
 [-5.92464,
  -5.46796,
  -5.70278,
  -5.70278,
  -4.31421,
  -5.70278,
  -5.70278,
  -5.70278,
  -5.70278,
  -5.70278,
  -5.58595,
  -5.63171,
  -5.43251,
  -5.65882,
  -4.60139,
  -5.37053,
  -5.21994,
  -5.88795,
  -5.15068,
  -6.8007,
  -5.8055,
  -5.42562,
 

In [11]:
matrix = []
for i in range(len(pairsname)):
    index1 = int(pairsname[i][1])
    index2 = int(pairsname[i][-1])
    row = [(score_values[index1-1][j]-score_values[index2-1][j]) for j in range(46)]
    matrix.append(row)
matrix    

[[5.42859,
  5.765000000000001,
  6.25324,
  6.1200399999999995,
  6.19388,
  6.14851,
  6.14851,
  6.14851,
  6.14851,
  6.14851,
  6.362690000000001,
  4.92746,
  6.1765,
  4.31864,
  6.1718399999999995,
  5.87383,
  6.11491,
  5.68418,
  6.300839999999999,
  5.56569,
  5.89164,
  6.13981,
  6.374029999999999,
  6.25483,
  5.96461,
  6.72011,
  6.13596,
  6.10223,
  5.48114,
  7.359389999999999,
  6.17004,
  5.76408,
  6.15471,
  5.27677,
  6.54492,
  5.98066,
  5.8715399999999995,
  6.40474,
  6.39997,
  6.23794,
  5.39759,
  6.038130000000001,
  6.14851,
  5.88135,
  5.75406,
  6.14844],
 [3.77846,
  3.67219,
  3.9779799999999996,
  4.102239999999999,
  3.65694,
  4.08336,
  4.08336,
  4.08336,
  4.08336,
  4.08336,
  3.8461600000000002,
  2.85172,
  3.50048,
  4.12249,
  3.6960800000000003,
  4.49183,
  4.19771,
  3.74751,
  3.6621399999999995,
  4.34673,
  3.58654,
  4.22596,
  4.86312,
  4.16134,
  4.03721,
  4.60644,
  4.02053,
  4.15511,
  2.8201799999999997,
  4.62602,
  4.29319,
  3.1819900000000003,
  4.083959999999999,
  4.142,
  4.08336,
  4.11304,
  3.1996599999999997,
  4.24621,
  4.50276,
  3.7092300000000002,
  3.4525200000000003,
  3.72847,
  4.08336,
  3.85765,
  3.72149,
  4.05163],
 [3.2693,
  3.68467,
  3.9650499999999997,
  3.8081199999999997,
  3.62425,
  3.96504,
  3.96504,
  3.96504,
  3.96504,
  3.96504,
  4.03371,
  2.8514299999999997,
  3.88952,
  2.4702100000000002,
  3.4767,
  3.54767,
  4.14272,
  3.72389,
  4.262029999999999,
  3.38405,
  4.0264299999999995,
  4.238359999999999,
  5.95463,
  5.39456,
  6.53632,
  3.48314,
  4.2277,
  4.301839999999999,
  3.28842,
  4.34142,
  4.0511800000000004,
  2.5422100000000003,
  4.07067,
  3.36754,
  4.35343,
  3.45232,
  3.0845899999999995,
  4.28856,
  5.49465,
  5.66052,
  3.3297600000000003,
  4.06763,
  3.96504,
  3.96935,
  3.7698,
  4.16309],
 [7.569100000000001,
  8.49453,
  8.796859999999999,
  9.11786,
  7.9790600000000005,
  8.902239999999999,
  8.902239999999999,
  8.902239999999999,
  8.902239999999999,
  8.902239999999999,
  8.47036,
  7.639340000000001,
  8.39457,
  6.95875,
  8.75824,
  9.72316,
  9.20392,
  8.68714,
  8.6101,
  8.42792,
  8.31669,
  8.74203,
  9.439910000000001,
  9.40508,
  8.785160000000001,
  9.38001,
  8.89542,
  8.98775,
  7.81239,
  9.546949999999999,
  8.98639,
  7.981310000000001,
  8.93402,
  8.80847,
  8.902239999999999,
  9.43259,
  8.6885,
  8.17008,
  7.89764,
  7.95688,
  8.0706,
  9.08628,
  8.902239999999999,
  8.88499,
  8.563320000000001,
  9.77707],
 [5.50058,
  5.71151,
  5.74923,
  6.22716,
  4.48437,
  5.85462,
  5.85462,
  5.85462,
  5.85462,
  5.85462,
  5.677090000000001,
  5.72385,
  5.26785,
  5.40596,
  4.83628,
  7.07987,
  5.446280000000001,
  5.94593,
  5.212210000000001,
  7.24344,
  5.29253,
  5.404800000000001,
  3.07038,
  4.997070000000001,
  3.2121000000000004,
  6.83526,
  5.57842,
  5.7376700000000005,
  5.214729999999999,
  6.04763,
  5.9707,
  6.5467900000000006,
  5.7807699999999995,
  6.42073,
  5.4662299999999995,
  7.0315,
  6.54379,
  4.79894,
  2.22271,
  3.0417300000000003,
  5.56682,
  6.0265,
  5.85462,
  5.33854,
  6.1117,
  6.1880999999999995],
 [4.2998,
  4.80986,
  4.83181,
  5.30974,
  4.3548100000000005,
  4.9372,
  4.9372,
  4.9372,
  4.9372,
  4.9372,
  4.43665,
  4.78791,
  4.505050000000001,
  4.4885399999999995,
  5.28154,
  6.17549,
  5.0612,
  4.9632499999999995,
  4.34807,
  5.04387,
  4.29026,
  4.5036700000000005,
  3.4852800000000004,
  4.01052,
  2.2488400000000004,
  5.89687,
  4.66772,
  4.685910000000001,
  4.52397,
  5.2055299999999995,
  4.93521,
  5.439100000000001,
  4.86335,
  5.44093,
  4.54881,
  5.98027,
  5.60391,
  3.88152,
  2.40299,
  2.29636,
  4.74084,
  5.01865,
  4.9372,
  4.91564,
  4.79352,
  5.61398],
 [4.26097,
  3.7742999999999998,
  3.82674,
  4.12463,
  4.26212,
  3.93213,
  3.93213,
  3.93213,
  3.93213,
  3.93213,
  3.4874400000000003,
  3.8141000000000003,
  3.49999,
  5.61789,
  4.17127,
  5.705170000000001,
  3.89483,
  3.

In [12]:
top_k_idx,_ = get_set_cover(matrix, 1)
top_k_idx

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


[15, 35, 45, 25, 19, 36, 33, 3, 41, 29]

In [13]:
features_to_change = tmp_test_data
if len(top_k_idx)<= feature_number:
    feature_number = len(top_k_idx)
features_to_change[:,top_k_idx[0:feature_number]] = expected_value[top_k_idx[0:feature_number]]
features_to_change

array([[0.052893  , 1.        , 0.75      , 0.13261472, 0.066225  ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.047634  , 1.        , 0.740506  , 1.        , 0.058539  ,
        0.14905815, 0.5       , 0.4       , 0.4       , 0.14922716,
        1.        , 1.        , 0.97451   , 1.        , 0.92924   ,
        0.13240912, 1.        , 0.829951  , 1.        , 0.15990731,
        0.768123  , 1.        , 1.        , 0.07848159, 1.        ,
        0.07429701, 0.54698628, 1.        , 0.998377  , 1.        ,
        0.333333  , 0.33050601, 0.        , 0.39691   , 0.447368  ,
        0.15198914],
       [0.004959  , 0.        , 0.25      , 0.13261472, 0.006623  ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.004971  , 0.        , 0.259494  , 0.521932  , 0.006639  ,
        0.14905815, 0.714286  , 0.7       , 0.        , 0.14922716,
        0.229604  , 0.237068  , 0.200021  , 0.063318  , 0.        ,
        0.13240912, 0.     

In [3]:
if __name__ == '__main__':
    #parameters to be set
    model_path = 'model/'
    model = model_path + 'LambdaMART_model.txt'
    
    
    # the path of data
    datapath = 'MQ2008/Fold1/'
    train_path = datapath + 'train.txt'
    test_path = datapath + 'test.txt'
    modelname = model.split("_")[0].split("/")[-1]

    # saving path and save files
    NDCGdata_path = 'NDCGdata/'
    temp_path = 'temp_data1/'
    

    # get train data and test data
    X_train, y_query_train, Query_train = get_microsoft_data(train_path)
    X_train = np.array(X_train)
    X_test, y_query_test, Query_test = get_microsoft_data(test_path)
    X_test = np.array(X_test)
    expected_value = np.mean(X_train, axis=0)

    # separate the test set
    test_data, test_y_query, test_Query, q_d_len = separate_set(y_query_test, X_test, Query_test)


    resultfile_NDCG = 'resultfile/' + 'result_{}_newmatrix_NDCG.txt'.format(modelname)
    resultfile_tau = 'resultfile/' + 'result_{}_newmatrix_tau.txt'.format(modelname)
    
    for threshold_flag in range(3):
        with Pool(10) as p:
            print(p.map(loop_query, [query_index for query_index in range(len(test_data))]))
        for feature_number in (5,10):
            NDCG_file_name = NDCGdata_path + 'SHAP_newmatrix_{}features_threshold{}'.format(feature_number,threshold_flag) + modelname + '.txt'
            NDCG_file_matrix = NDCGdata_path + 'SHAP_newmatrix_matrix_{}features_threshold{}'.format(feature_number,threshold_flag)  + modelname + '.txt'
            ranklist_file = NDCGdata_path + 'ranklist_SHAP_newmatrix_{}features_threshold{}'.format(feature_number,threshold_flag)  + modelname + '.txt'
            rerank_ndcg(NDCG_file_name)
            NDCG =  write_average(NDCG_file_name)
            rerank_ndcg(ranklist_file)
            rerank_ndcg(NDCG_file_matrix)
            tau = write_tau(NDCG_file_name)
            with open(resultfile_NDCG, 'a') as NDCG_result:
                NDCG_result_line  = str(NDCG) + "\n"
                NDCG_result.write(NDCG_result_line)
            with open(resultfile_tau,'a') as tau_result:
                tau_result_line  = str(tau) + "\n" 
                tau_result.write(tau_result_line)

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]


Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an e

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an e

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an e

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an e

...

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an e

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an e

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an e

Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an e

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
